In [1]:
import torch
import gc
import os
from tqdm import tqdm
import numpy as np
import PIL.Image as pilimg
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms


In [2]:
def train(epoch_size , train_data , model , device):
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001 , weight_decay=1e-5)
    model.to(device)
    
    
    
    
    PATH = r"./models/resnet_224.pth"
    pre_loss = 999
    for epoch in range(epoch_size):   # 데이터셋을 수차례 반복합니다.
        running_loss = 0.0
        count = 0
        for img_i ,label_i in train_data:
            img_i ,label_i = img_i.view(-1,3,224,224).to(device), label_i.to(device)
            optimizer.zero_grad()
            outputs = model(img_i)
            loss = criterion(outputs , label_i.view(-1).long())
            loss.backward()
            optimizer.step()
            count +=1
            running_loss += loss.item()
            if count % 30 == 0:
                new_loss = (running_loss / 30)
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, count + 1, new_loss))

                if pre_loss > new_loss:
                    pre_loss = new_loss
                    torch.save(model.state_dict(), PATH)
                    print("Model svae")
                running_loss = 0.0

    print('Finished Training')
    
def test(test_data , model , device):
    correct = 0
    total = 0
    true_positive = 0
    false_positive = 0
    false_negative = 0
    model.to(device)
    for img_i ,label_i in test_data :
        img_i ,label_i = img_i.view(-1,3,224,224).to(device), label_i.view(50).to(device)        
        outputs = model(img_i)
        _, predicted = torch.max(outputs, 1)
        total += label_i.size(0)

        label_temp = (label_i.long() == 0)
        label_temp2 = (label_i.long() == 1)
        pre_temp = (predicted == 0) # positive
        for i in range(0,len(pre_temp)):
            if (pre_temp[i]==True): # positive
                if(label_temp[i]==True):
                    true_positive +=1 # 
                else:
                    false_positive +=1
            else: # negative
                if((label_temp2[i]==False)):
                    false_negative +=1
                        
        correct += (predicted == label_i.long()).sum().item()
    print('Accuracy of the network on the 13200 test images: %d %%' % (100 * correct / total))
    print('Precision of the network on the 13200 test images: %d %%' % (100 * true_positive / (true_positive+false_positive)))
    print('Recall of the network on the 13200 test images: %d %%' % (100 * true_positive / (true_positive+false_negative)))
    print("True_positive : " , true_positive)
    print("False_positive : " , false_positive)
    print("False_negative : " , false_negative)
    print("Correct : " , correct)
    print("Total : " ,total)


In [3]:
train_dataset = dset.ImageFolder(root=r"./img/train",
                           transform=transforms.Compose([
                               transforms.Scale(224),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)),
                           ]))

test_dataset = dset.ImageFolder(root=r"./img/test",
                           transform=transforms.Compose([
                               transforms.Scale(224),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)),
                           ]))


train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=50,
                                         shuffle=True,)


test_dataloader = torch.utils.data.DataLoader(test_dataset , batch_size=50,
                                         shuffle=True,)


C:\Users\hyeongy\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [4]:
# from torchvision.transforms import ToPILImage
# print(train_dataloader)
# to_img = ToPILImage()

# print(len(train_dataloader))
# for i, data in enumerate(train_dataloader):
#     img = data[0][0,:]
#     plt.imshow(to_img(img))
#     plt.figure()

In [5]:
print(len(train_dataloader))
print(len(test_dataloader))

1056
264


In [6]:
model = models.resnet50()
model.fc = nn.Linear(2048, 2)
model.train()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train(7, train_dataloader , model , device) # epoch_size , train_data , model , device


[1,    31] loss: 0.767
Model svae
[1,    61] loss: 0.478
Model svae
[1,    91] loss: 0.454
Model svae
[1,   121] loss: 0.464
[1,   151] loss: 0.425
Model svae
[1,   181] loss: 0.408
Model svae
[1,   211] loss: 0.381
Model svae
[1,   241] loss: 0.406
[1,   271] loss: 0.356
Model svae
[1,   301] loss: 0.320
Model svae
[1,   331] loss: 0.328
[1,   361] loss: 0.401
[1,   391] loss: 0.318
Model svae
[1,   421] loss: 0.309
Model svae
[1,   451] loss: 0.286
Model svae
[1,   481] loss: 0.262
Model svae
[1,   511] loss: 0.291
[1,   541] loss: 0.247
Model svae
[1,   571] loss: 0.215
Model svae
[1,   601] loss: 0.258
[1,   631] loss: 0.226
[1,   661] loss: 0.219
[1,   691] loss: 0.226
[1,   721] loss: 0.194
Model svae
[1,   751] loss: 0.222
[1,   781] loss: 0.205
[1,   811] loss: 0.162
Model svae
[1,   841] loss: 0.214
[1,   871] loss: 0.206
[1,   901] loss: 0.209
[1,   931] loss: 0.194
[1,   961] loss: 0.167
[1,   991] loss: 0.172
[1,  1021] loss: 0.173
[1,  1051] loss: 0.131
Model svae
[2,    3

In [9]:
# PATH = r'./models/googlenet2_224.pth'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = models.googlenet()
# model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [8]:
model.eval()
test(train_dataloader , model , device)

Accuracy of the network on the 13200 test images: 95 %
Precision of the network on the 13200 test images: 99 %
Recall of the network on the 13200 test images: 92 %
True_positive :  24343
False_positive :  60
False_negative :  2057
Correct :  50683
Total :  52800


In [ ]:
# model.train()
# train(5, train_data_loader , model , device)